In [1]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz
import time

# Project libraries
import constants

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"
# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
# my_app_key = constants.API_KEY_DEMO #demo
my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [2]:
## TODO
liability_amount = 1500

In [3]:
#######################################
# Filter out only greyhoud races
#######################################
gh_racing_id = constants.GH_RACING_ID #Greyhound racing ID
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
    }
)
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

# Create a DataFrame with all the events by iterating over each event object
gb_gh_events_df = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_gh_events],
    'Event ID': [event_object.event.id for event_object in gb_gh_events],
    'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
    'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
    'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
    'Market Count': [event_object.market_count for event_object in gb_gh_events]
})

gb_gh_events_df

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count


In [4]:
#######################################
#TODO: extract a list of IDs for the forecast markets
#######################################
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


([], [])

In [5]:
#######################################
## Filter out the next 5 upcoming games
#######################################
market_catalogue_filter = filters.market_filter(
    event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    market_projection= ['MARKET_START_TIME', 'EVENT'],
    max_results='5',
    sort='FIRST_TO_START'
)

market_types_venueOfTheDay_df = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
    'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
    
})

market_types_venueOfTheDay_df

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Next Conservative Leader.,1.160663234,2016-12-14 17:00:00,4.032503e+05,None
1,Exit Dates - Boris Johnson 2,1.160740937,2016-12-14 17:00:00,2.151179e+06,None
2,Will Rishi Sunak be the next Conservative leader?,1.194425281,2016-12-14 17:00:00,8.299821e+03,None
3,Year Keir Starmer replaced as Labour Leader,1.183324545,2016-12-14 17:00:00,4.017826e+04,None
4,Winner 2022/23,1.199751258,2018-08-11 14:00:00,1.647627e+03,None


In [6]:
#######################################
## Check the start time of the next game
## and sleep until 15 seconds before the game
#######################################
from utils import get_next_market

time_gap, myRaceID, myRaceVenue = get_next_market(market_catalogues)
assert myRaceID == 0, "ERROR: myRaceID = 0" #TODO: remove in production

if(time_gap > datetime.timedelta(seconds=constants.PREBET_DELAY)):
    time_to_sleep = (time_gap - datetime.timedelta(seconds=constants.PREBET_DELAY)).seconds
    print("Sleeping for " + str(time_to_sleep) + " seconds")
    time.sleep(time_to_sleep)
    print("Sleeping done")
else:
    print("Don't need to sleep")

##TODO Choose the liability amount



Time Now: 
2022-06-22 21:56:06
Market Start Time: 2018-08-11 14:00:00+00:00
Don't need to sleep


In [ ]:
#######################################
# Get a list of lay options
#######################################
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]
runners_df = process_runner_books(market_book.runners)

#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)
# print(lay_options_ids)

#######################################
# Get odds from NEDS
#######################################
from utils import choose_lay_option

lay_selection_index = choose_lay_option(myRaceVenue)

if(lay_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    assert()
    
elif(lay_selection_index == -2):
    print("ERROR!!! less than 6 runners")
    assert()

elif(lay_selection_index == -3):
    print("ERROR!!! NEDS API failed")
    assert()
else:
    print("Lay option found successfully")
    print(lay_selection_index)

lay_selection_id = lay_options_ids[lay_selection_index]
print("lay_selection_id = " + str(lay_selection_id))
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

#######################################
# Choose order type (limit order or market_on_close order - choose the latter)
#######################################
# order_filter = filters.limit_order(
#     price=fav_price,
#     persistence_type='LAPSE',
#     bet_target_type='PAYOUT',
#     bet_target_size=liability_amount # use this if i want $1500 as liability
# )

order_filter = filters.market_on_close_order(
    liability=liability_amount
)
print(order_filter)

#######################################
# Create a place instruction filter
#######################################
instructions_filter = filters.place_instruction(
    selection_id = str(lay_selection_id),
    side="LAY",
    ## fixed price order
    # order_type = "LIMIT",
    # limit_order=limit_order_filter

    ## flexible price order
    order_type = "MARKET_ON_CLOSE",
    market_on_close_order= order_filter
)
print(instructions_filter)

In [ ]:
#######################################
# Place the order
#######################################
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [11]:
#######################################
# List the cleared orders
#######################################
# myRaceID = 1.200362941
time.sleep(5)
current_orders = trading.betting.list_current_orders(
    customer_strategy_refs=['back_the_fav']).__dict__
data = pd.DataFrame(current_orders._data['currentOrders']).head()
print(data)

cleared_orders = trading.betting.list_cleared_orders(
    bet_status="SETTLED",
    market_ids=[myRaceID])

# 
# Create a DataFrame from the orders
data = pd.DataFrame(cleared_orders._data['clearedOrders']).head()
print(data)

""
